In [0]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import re
import time
import os
import numpy as np
import pandas as pd
from itertools import product
import concurrent.futures

In [0]:
URL = "https://wonder.cdc.gov/cancer-v2021.HTML"

In [0]:
def create_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")  
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    service = Service()
    driver = webdriver.Chrome(options=chrome_options, service=service)
    return driver


In [0]:
MSA_SELECT_ID = "SD198.V3"
MSA_OPTION_ID = "RO_locationD198.V3"
YEAR_SELECT_ID = "SD198.V1"
SEX_SELECT_ID = "SD198.V9"
AGE_GROUP_SELECT_ID = "SD198.V5"
ETHNICITY_SELECT_ID = "SD198.V6"
RACE_SELECT_ID = "SD198.V4"
CANCER_SITES_SELECT_ID = "SD198.V8"
FOOTER_BUTTONS_CLASS = "footer-buttons"
SEND_BUTTON_VALUE = "Send"

In [0]:
DESELECT_DICT = {
    MSA_SELECT_ID: "10900",
    YEAR_SELECT_ID: "2002",
    SEX_SELECT_ID: "F",
    AGE_GROUP_SELECT_ID: "15-19",
    ETHNICITY_SELECT_ID: "2186-5",
    RACE_SELECT_ID: "2054-5",
    CANCER_SITES_SELECT_ID: "20030"
}

In [0]:
def get_select_options(select_id, driver):
    select = driver.find_element(By.ID, select_id)
    options = select.find_elements(By.XPATH, "./*")
    value_options = {option.get_property('value'): option for option in options}
    name_values = {re.sub(r'\(\d+\)', '', option.text).strip(): option.get_property('value') for option in options}
    return name_values 

In [0]:
def select_msa_option(driver):
    msa_option = driver.find_element(By.ID, MSA_OPTION_ID)
    msa_option.click()

In [0]:
driver = create_driver()
driver.get(URL)

In [0]:
select_msa_option(driver)
msa_name_values = get_select_options(MSA_SELECT_ID, driver)
year_name_values = get_select_options(YEAR_SELECT_ID, driver)
sex_name_values = get_select_options(SEX_SELECT_ID, driver)
age_group_name_values = get_select_options(AGE_GROUP_SELECT_ID, driver)
ethnicity_name_values = get_select_options(ETHNICITY_SELECT_ID, driver)
race_name_values = get_select_options(RACE_SELECT_ID, driver)
cancer_site_name_values = get_select_options(CANCER_SITES_SELECT_ID, driver)


In [0]:
driver.quit()

In [0]:
msa_name_values.pop('The United States')
year_name_values.pop("All Years")
sex_name_values.pop("All Sexes")
age_group_name_values.pop("All Ages")
ethnicity_name_values.pop("All Ethnicities")
race_name_values.pop("All Races")
cancer_site_name_values = {"Gallbladder": cancer_site_name_values["Gallbladder"], "Lung and Bronchus": cancer_site_name_values["Lung and Bronchus"]}


In [0]:
total_expected_rows = len(msa_name_values) * len(year_name_values) * len(sex_name_values) * len(age_group_name_values) * len(ethnicity_name_values) * len(race_name_values) * len(cancer_site_name_values)

In [0]:
data = pd.DataFrame(list(product(msa_name_values, year_name_values, sex_name_values, age_group_name_values, ethnicity_name_values, race_name_values, cancer_site_name_values)), columns=['msa', 'year', 'sex', 'age_group', 'ethnicity', 'race', 'cancer_site'])

In [0]:
n_buckets = int(data.shape[0] / 100)
data['bucket'] = np.random.permutation(data.shape[0]) % n_buckets

In [0]:
data['index'] = list(range(data.shape[0]))
data['count'] = np.nan

In [0]:
# spark.sql("""create or replace table !!!cancer_incidence!!! (
#   msa string,
#   year string,
#   sex string,
#   age_group string,
#   ethnicity string,
#   race string,
#   cancer_site string,
#   bucket string,
#   index int,
#   count float
# )""")

In [0]:
def select_msa_option(driver):
    msa_option = driver.find_element(By.ID, MSA_OPTION_ID)
    msa_option.click()

def select_send(driver):
    footer_buttons = driver.find_element(By.CLASS_NAME, FOOTER_BUTTONS_CLASS)
    send_button = footer_buttons.find_element(By.XPATH, f'//*[@value="{SEND_BUTTON_VALUE}"]')
    send_button.click()

def select_option(select_id, option_value, driver):
    select = driver.find_element(By.ID, select_id)
    option = driver.find_element(By.XPATH, f'//*[@value="{option_value}"]')
    deselect_option = driver.find_element(By.XPATH, f'//*[@value="{DESELECT_DICT[select_id]}"]')

    select.click()
    deselect_option.click()
    option.click()


def one_download(row, driver):
    msa, year, sex, age_group, ethnicity, race, cancer_site = row
    select_msa_option(driver)
    select_option(MSA_SELECT_ID, msa_name_values[msa], driver)
    select_option(YEAR_SELECT_ID, year_name_values[year], driver)
    select_option(SEX_SELECT_ID, sex_name_values[sex], driver)
    select_option(AGE_GROUP_SELECT_ID, age_group_name_values[age_group], driver)
    select_option(ETHNICITY_SELECT_ID, ethnicity_name_values[ethnicity], driver)
    select_option(RACE_SELECT_ID, race_name_values[race], driver)
    select_option(CANCER_SITES_SELECT_ID, cancer_site_name_values[cancer_site], driver)
    select_send(driver)

    try:
        driver.find_element(By.ID, 'error-messages') 
        count = 0
    except Exception as err:
        table = driver.find_element(By.CLASS_NAME, 'response-form')
        count = int(table.find_element(By.TAG_NAME, 'td').text)

    driver.get(URL)
    return count

def download_multiple(bucket):
    print(f'Processing {bucket}')
    df = data[data['bucket'] == bucket].copy()
    driver = create_driver()
    driver.get(URL)
    df['count'] = df.drop(columns=['index', 'bucket', 'count']).apply(one_download, driver=driver, axis=1)
    spark.createDataFrame(df).createOrReplaceTempView("new_insert")

    spark.sql("""
           insert into cancer_incidence  
           select * from new_insert
    """)
    driver.quit()
    print(f'Finished {bucket}')



In [0]:
existing_buckets = spark.sql("select distinct bucket from default.cancer_incidence").toPandas()

In [0]:
155 * 100

In [0]:
failed = []
for bucket in data['bucket'].unique():
  try:
    download_multiple(bucket)
  except Exception as err:
    failed.append([bucket, err])

In [0]:
failed